In [1]:
import scanpy as sc
import anndata as ad

In [2]:
# process atac_mtg
n_top_genes = 5000

atac_mtg = sc.read_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_MTG_ATACseq_final-nuclei.2024-02-13.h5ad')
# remove mislabeled obs
atac_mtg = atac_mtg[~atac_mtg.obs['Supertype'].isin(['n_genes', 'fraction_mito', 'doublet_score','cluster_fraction_mito_flag','cluster_doublet_score_flag','cluster_fraction_ribo_flag', 'cluster_n_genes_flag'])]


In [3]:
atac_mtg

View of AnnData object with n_obs × n_vars = 551322 × 218882
    obs: 'sample_id', 'Neurotypical reference', 'Donor ID', 'Organism', 'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)', 'Race (choice=Black/ African American)', 'Race (choice=Asian)', 'Race (choice=American Indian/ Alaska Native)', 'Race (choice=Native Hawaiian or Pacific Islander)', 'Race (choice=Unknown or unreported)', 'Race (choice=Other)', 'specify other race', 'Hispanic/Latino', 'Highest level of education', 'Years of education', 'PMI', 'Fresh Brain Weight', 'Brain pH', 'Overall AD neuropathological Change', 'Thal', 'Braak', 'CERAD score', 'Overall CAA Score', 'Highest Lewy Body Disease', 'Total Microinfarcts (not observed grossly)', 'Total microinfarcts in screening sections', 'Atherosclerosis', 'Arteriolosclerosis', 'LATE', 'Cognitive Status', 'Last CASI Score', 'Interval from last CASI in months', 'Last MMSE Score', 'Interval from last MMSE in months', 'Last MOCA Score', 'Interval from last MO

In [6]:
# all donors in mtg have atac
seaad_mtg = sc.read_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_MTG_RNAseq_final-nuclei.2024-02-13_count_hvg_aligned.h5ad')
print(len(set(seaad_mtg.obs['Donor_ID'].unique()).intersection(atac_mtg.obs['Donor ID'].unique())))

84


In [7]:
atac_mtg.obs.rename(columns={'Donor ID': 'Donor_ID'}, inplace=True)

In [10]:
# number of cells per donor ranges from 863 to 20015
atac_mtg.obs.groupby('Donor_ID').count().sort_values(by = 'sample_id')

/tmp/ipykernel_113660/1445347638.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  atac_mtg.obs.groupby('Donor_ID').count().sort_values(by = 'sample_id')


,sample_id,Neurotypical reference,Organism,Brain Region,Sex,Gender,Age at Death,Race (choice=White),Race (choice=Black/ African American),Race (choice=Asian),...,Class,Subclass confidence,Subclass,Supertype confidence,Supertype (non-expanded),Supertype,RNA Quality Control Score,Quality Control Clusters,Continuous Pseudo-progression Score,Severely Affected Donor
Donor_ID,,,,,,,,,,,,,,,,,,,,,
H21.33.020,863,863,863,863,863,863,863,863,863,863,...,863,863,863,0,0,0,863,863,863,863
H21.33.034,983,983,983,983,983,983,983,983,983,983,...,983,983,983,0,0,0,983,983,983,983
H21.33.039,1023,1023,1023,1023,1023,1023,1023,1023,1023,1023,...,1023,1023,1023,0,0,0,1023,1023,1023,1023
H20.33.033,1026,1026,1026,1026,1026,1026,1026,1026,1026,1026,...,1026,1026,1026,0,0,0,1026,1026,1026,1026
H21.33.026,1181,1181,1181,1181,1181,1181,1181,1181,1181,1181,...,1181,1181,1181,0,0,0,1181,1181,1181,1181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H20.33.027,15283,15283,15283,15283,15283,15283,15283,15283,15283,15283,...,15283,15283,15283,5893,5893,5893,15283,15283,15283,15283
H20.33.005,16221,16221,16221,16221,16221,16221,16221,16221,16221,16221,...,16221,16221,16221,5692,5692,5692,16221,16221,16221,16221
H20.33.043,16956,16956,16956,16956,16956,16956,16956,16956,16956,16956,...,16956,16956,16956,8070,8070,8070,16956,16956,16956,16956


In [11]:
sc.pp.normalize_per_cell(atac_mtg, counts_per_cell_after=1e4)
sc.pp.log1p(atac_mtg)
sc.pp.highly_variable_genes(atac_mtg, n_top_genes=n_top_genes, subset=True)

/home/icb/zihe.zheng/miniconda3/envs/scanpy_dask/lib/python3.13/site-packages/scanpy/preprocessing/_simple.py:578: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell


In [12]:
atac_mtg.write_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_MTG_ATACseq_final-nuclei.2024-02-13_hvg_aligned.h5ad')
